In [9]:
import os
import tensorflow as tf
DATA_DIR = "D:\Smart parking\Data\PKLot\PKLotSegm_sample"

def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_data = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR, image_size=(71,71), batch_size=16, validation_split=0.2,
    subset="training", seed=123, 
)
train_data = train_data.map(process)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR, image_size=(71,71), batch_size=16, validation_split=0.2,
    subset="validation", seed=123, 
)
test_data = test_data.map(process)

Found 10000 files belonging to 2 classes.
Using 8000 files for training.
Found 10000 files belonging to 2 classes.
Using 2000 files for validation.


In [7]:
import tensorflow as tf
xception = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(71,71,3),
    pooling=None,
    classes=1,
    classifier_activation="softmax",
)

In [8]:
xception.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 71, 71, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 35, 35, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 35, 35, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 35, 35, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [15]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
x = xception.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
preds = Dense(1, activation = 'sigmoid')(x)

model = Model(inputs=xception.input, outputs=preds)

for layer in xception.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 71, 71, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 35, 35, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 35, 35, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 35, 35, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [16]:
with tf.device('/GPU:0'):
    model.fit(train_data, batch_size=16, epochs=150)

Epoch 1/150
500/500 [==============================] - 59s 54ms/step - loss: 0.5695 - accuracy: 0.9739
Epoch 2/150
500/500 [==============================] - 24s 48ms/step - loss: 0.1713 - accuracy: 0.9900
Epoch 3/150
500/500 [==============================] - 26s 52ms/step - loss: 0.1145 - accuracy: 0.9930
Epoch 4/150
500/500 [==============================] - 25s 51ms/step - loss: 0.0705 - accuracy: 0.9959
Epoch 5/150
500/500 [==============================] - 26s 52ms/step - loss: 0.0400 - accuracy: 0.9980
Epoch 6/150
500/500 [==============================] - 23s 46ms/step - loss: 0.0348 - accuracy: 0.9974
Epoch 7/150
500/500 [==============================] - 17s 34ms/step - loss: 0.0182 - accuracy: 0.9979
Epoch 8/150
500/500 [==============================] - 25s 51ms/step - loss: 0.0116 - accuracy: 0.9990
Epoch 9/150
500/500 [==============================] - 20s 41ms/step - loss: 0.0326 - accuracy: 0.9981
Epoch 10/150
500/500 [==============================] - 22s 45ms/step - l

In [20]:
model.save('Xception.pt')

INFO:tensorflow:Assets written to: Xception.pt\assets


c:\Users\Usuario\Documents\proyectos\smart_parking\carnet\.venv\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [21]:
vgg16 = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(71,71,3),
    pooling=None,
    classes=1
)

In [22]:
x = vgg16.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
preds = Dense(1, activation = 'sigmoid')(x)

model2 = Model(inputs=vgg16.input, outputs=preds)

for layer in vgg16.layers:
    layer.trainable = False

model2.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['accuracy'])

model2.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 71, 71, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 71, 71, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 71, 71, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 35, 35, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 35, 35, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 35, 35, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 17, 17, 128)       0   

In [23]:
with tf.device('/GPU:0'):
    model2.fit(train_data, batch_size=16, epochs=150)

Epoch 1/150
500/500 [==============================] - 21s 30ms/step - loss: 0.1025 - accuracy: 0.9685
Epoch 2/150
500/500 [==============================] - 6s 12ms/step - loss: 0.0477 - accuracy: 0.9868
Epoch 3/150
500/500 [==============================] - 6s 11ms/step - loss: 0.0338 - accuracy: 0.9904
Epoch 4/150
500/500 [==============================] - 6s 11ms/step - loss: 0.0282 - accuracy: 0.9925
Epoch 5/150
500/500 [==============================] - 5s 11ms/step - loss: 0.0228 - accuracy: 0.9937
Epoch 6/150
500/500 [==============================] - 5s 11ms/step - loss: 0.0223 - accuracy: 0.9951
Epoch 7/150
500/500 [==============================] - 5s 11ms/step - loss: 0.0175 - accuracy: 0.9958
Epoch 8/150
500/500 [==============================] - 5s 11ms/step - loss: 0.0156 - accuracy: 0.9961
Epoch 9/150
500/500 [==============================] - 5s 11ms/step - loss: 0.0141 - accuracy: 0.9969
Epoch 10/150
500/500 [==============================] - 5s 11ms/step - loss: 0.01

In [24]:
model2.save('VGG16.pt')

INFO:tensorflow:Assets written to: VGG16.pt\assets


In [25]:
del model
del model2

# Efficient Net


In [26]:
effnet = tf.keras.applications.EfficientNetB3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(71,71,3),
    pooling=None,
    classes=1
)

43950080/43941136 [==============================] - 5s 0us/step


In [27]:
x = effnet.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
preds = Dense(1, activation = 'sigmoid')(x)

model3 = Model(inputs=effnet.input, outputs=preds)

for layer in effnet.layers:
    layer.trainable = False

model3.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['accuracy'])

model3.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 71, 71, 3)]  0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 71, 71, 3)    0           input_5[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 71, 71, 3)    7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 73, 73, 3)    0           normalization[0][0]              
____________________________________________________________________________________________

In [28]:
with tf.device('/GPU:0'):
    model3.fit(train_data, batch_size=16, epochs=150)

Epoch 1/150
500/500 [==============================] - 23s 33ms/step - loss: 1.9965 - accuracy: 0.5063
Epoch 2/150
500/500 [==============================] - 21s 42ms/step - loss: 0.7030 - accuracy: 0.5026
Epoch 3/150
500/500 [==============================] - 22s 44ms/step - loss: 0.6957 - accuracy: 0.49791s -
Epoch 4/150
500/500 [==============================] - 22s 45ms/step - loss: 0.6956 - accuracy: 0.4915
Epoch 5/150
500/500 [==============================] - 23s 45ms/step - loss: 0.6951 - accuracy: 0.4947
Epoch 6/150
500/500 [==============================] - 23s 45ms/step - loss: 0.6944 - accuracy: 0.4916
Epoch 7/150
500/500 [==============================] - 23s 45ms/step - loss: 0.6935 - accuracy: 0.4945
Epoch 8/150
500/500 [==============================] - 22s 44ms/step - loss: 0.6934 - accuracy: 0.4899
Epoch 9/150
500/500 [==============================] - 22s 44ms/step - loss: 0.6932 - accuracy: 0.4938
Epoch 10/150
500/500 [==============================] - 20s 40ms/step

In [29]:
model3.save('effnet.pt')

INFO:tensorflow:Assets written to: effnet.pt\assets


c:\Users\Usuario\Documents\proyectos\smart_parking\carnet\.venv\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [36]:
DATA_DIR = "D:\Smart parking\Data\Data_Marko_cropped"
marko_data = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR, image_size=(71,71), batch_size=1, seed=123
)
marko_data = marko_data.map(process)

Found 2088 files belonging to 2 classes.


In [42]:
model = tf.keras.models.load_model('VGG16.pt')

In [43]:
scores = model.evaluate(test_data, verbose=1)

125/125 [==============================] - 2s 14ms/step - loss: 0.1644 - accuracy: 0.9925


In [44]:
scores = model.evaluate(marko_data, verbose=1)

2088/2088 [==============================] - 11s 5ms/step - loss: 14.4667 - accuracy: 0.8041


In [46]:
model = tf.keras.models.load_model('Xception.pt')

In [47]:
scores = model.evaluate(test_data, verbose=1)

125/125 [==============================] - 2s 14ms/step - loss: 0.2135 - accuracy: 0.9940


In [48]:
scores = model.evaluate(marko_data, verbose=1)

2088/2088 [==============================] - 26s 12ms/step - loss: 10.9067 - accuracy: 0.8736


In [51]:
model = tf.keras.models.load_model('effnet.pt')

In [52]:
scores = model.evaluate(test_data, verbose=1)

125/125 [==============================] - 5s 29ms/step - loss: 0.6932 - accuracy: 0.4950


In [53]:
scores = model.evaluate(marko_data, verbose=1)

2088/2088 [==============================] - 55s 26ms/step - loss: 0.6923 - accuracy: 0.6566


# Xception transfer learning


In [6]:
import tensorflow as tf
transfer_model = tf.keras.models.load_model('Xception.pt')

In [8]:
DATA_DIR = "D:\Smart parking\Data\CTIC_Parking_dataset"
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label
data_train = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR, image_size=(71,71), batch_size=16, seed=123, validation_split = 0.2, subset="training"
)
data_train = data_train.map(process)

data_test = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR, image_size=(71,71), batch_size=16, seed=123, validation_split = 0.2, subset="validation"
)
data_test = data_test.map(process)

Found 2984 files belonging to 2 classes.
Using 2388 files for training.
Found 2984 files belonging to 2 classes.
Using 596 files for validation.


In [9]:
with tf.device('/GPU:0'):
    transfer_model.fit(data_train, batch_size=16, epochs=500)

Epoch 1/500
150/150 [==============================] - 6s 20ms/step - loss: 2.4904 - accuracy: 0.9631
Epoch 2/500
150/150 [==============================] - 3s 19ms/step - loss: 0.4423 - accuracy: 0.9824
Epoch 3/500
150/150 [==============================] - 3s 19ms/step - loss: 0.2563 - accuracy: 0.9887
Epoch 4/500
150/150 [==============================] - 3s 19ms/step - loss: 0.1854 - accuracy: 0.9891
Epoch 5/500
150/150 [==============================] - 3s 19ms/step - loss: 0.1803 - accuracy: 0.9925
Epoch 6/500
150/150 [==============================] - 3s 19ms/step - loss: 0.1081 - accuracy: 0.9950
Epoch 7/500
150/150 [==============================] - 3s 19ms/step - loss: 0.0559 - accuracy: 0.9958
Epoch 8/500
150/150 [==============================] - 3s 19ms/step - loss: 0.0789 - accuracy: 0.9966
Epoch 9/500
150/150 [==============================] - 3s 19ms/step - loss: 0.0727 - accuracy: 0.9962
Epoch 10/500
150/150 [==============================] - 3s 19ms/step - loss: 0.074

In [10]:
scores = transfer_model.evaluate(data_test, verbose=1)

38/38 [==============================] - 1s 17ms/step - loss: 0.8155 - accuracy: 0.9782


In [11]:
transfer_model.save('Xception_transfer.pt')

INFO:tensorflow:Assets written to: Xception_transfer.pt\assets


c:\Users\Usuario\Documents\proyectos\smart_parking\carnet\.venv\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
